#Required

In [ ]:
!pip install pymp-pypi

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pymp

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
X_data = []
y_data = []

for name in glob.glob("/content/drive/MyDrive/Colab Notebooks/HPC Project/combined stocks/*.csv"):
    stocks = pd.read_csv(name)
    stocks.dropna(inplace = True)
    stocks["Date"] = pd.DatetimeIndex(stocks["Date"])
    stocks["Day"] = (stocks["Date"] - stocks["Date"][0])/np.timedelta64(1,"D")
    stocks.drop(labels=["Date","Volume"],axis=1,inplace=True)
    X_data.append(stocks.drop(labels=["Close"],axis=1).values)
    y_data.append(stocks["Close"].values)

In [ ]:
X_data = pymp.shared.list(X_data)
y_data = pymp.shared.list(y_data)

In [ ]:
X_data

<ListProxy object, typeid 'list' at 0x7fb01c77e490>

#KNN

In [ ]:
class KNN():

    def __init__(self):

        self.X_train = []
        self.y_train = []

    def euclidian(self,x1,x2):

        difference =  np.subtract(x1,x2)
        square_difference = np.power(difference,2)
        sum_square_difference = np.sum(square_difference)

        return sum_square_difference**(0.5)

    def fit(self,X,y):

        self.X_train = X
        self.y_train = y

    def predict(self,X_test,k):

        y_pred = []

        for i in range(X_test.shape[0]):

            distances = []

            for j in range(self.X_train.shape[0]):

                distances.append(self.euclidian(X_train[j],X_test[i]))

            indeces = np.argsort(np.array(distances))
            
            sol = 0

            for K in range(k):

                sol += self.y_train[indeces[K]]

            y_pred.append(sol/k)

        return y_pred

#Compute

In [ ]:
split = 0.2

In [ ]:
X_data[1:5]

[]

In [ ]:
%%time
with pymp.Parallel(num_threads=2) as p:
    for i in p.range(200):
        X_train,X_test,y_train,y_test = train_test_split(X_data[i],y_data[i],test_size=split)
        model = KNN()
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test,10)